In [1]:
import pandas as pd
import sqlalchemy 
from sqlalchemy import text
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
from dateutil import relativedelta
import os
import sys

sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

In [2]:
##VOLTAR PRA 15 ANTES DE RODAR, TALVEZ SEJA MELHOR RODAR DO D(0) MESMO
data = datetime.today().strftime("%d-%m-%Y")
diretorio_base = "C:\\Scripts\\relatórios\\"
horario = os.listdir("{}{}".format(diretorio_base, data))[0]
quinze_dias = (datetime.today() - timedelta(days=7)).strftime("%Y-%m-%d")

#### CAPTAÇÃO

In [3]:
captacao = pd.read_excel(diretorio_base+"%s\\%s\\nnm.xlsx" % (data, horario), header=2)
for coluna in captacao.columns:
    captacao.rename(columns={coluna:coluna.upper()}, inplace=True)
    
captacao['CONTA'] = captacao['CONTA'].astype(str)
captacao.rename(columns={"ASSESSOR":'Assessor'}, inplace=True)
captacao.loc[captacao['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "Rodrigo de Mello D’Elia"
captacao['Assessor'] = captacao['Assessor'].apply(lambda nome: str(nome).upper())
captacao = captacao.loc[:,["CONTA", "MERCADO", "DESCRIÇÃO", "ATIVO",
                           "CAPTAÇÃO", "DATA", "TIPO", "Assessor"]]

##### OFFSHORE

In [4]:
contas_offshore = pd.read_excel(r"C:\Scripts\historico_auc\AuC Offshore.xlsx", sheet_name='AuC Offshore')
contas_offshore = contas_offshore.loc[:,["Conta", "Assessor"]]

In [5]:
offshore = pd.read_excel(r"C:\Scripts\historico_nnm\NNM Offshore.xlsx", sheet_name="NNM Offshore")

offshore['Conta'] = offshore['Conta'].astype(str)
offshore = offshore.merge(contas_offshore, on='Conta', how='left')
offshore = offshore[~offshore['Conta'].isnull()]
offshore.rename(columns={"Data NNM":"DATA", "Conta":"CONTA", "NNM BRL":"CAPTAÇÃO"}, inplace=True)
offshore.drop(["NNM USD"], axis=1, inplace=True)
offshore['Assessor'] = [str(nome).upper() for nome in offshore['Assessor']]
offshore['DATA'] = pd.to_datetime(offshore['DATA'], format="%d/%m/%Y")
offshore['DESCRIÇÃO'] = "OFFSHORE"

In [6]:
captacao = pd.concat([offshore, captacao])
captacao['Assessor'] = captacao['Assessor'].astype(str).apply(lambda nome: nome.upper())

In [7]:
captacao

,DATA,CONTA,CAPTAÇÃO,Assessor,DESCRIÇÃO,MERCADO,ATIVO,TIPO
0,2022-01-31,TL7003471,-1.565587e+03,NAN,OFFSHORE,NaN,NaN,NaN
1,2022-01-31,TL7003687,-1.034555e+03,NAN,OFFSHORE,NaN,NaN,NaN
2,2022-01-31,TL7010013,2.211178e+06,NAN,OFFSHORE,NaN,NaN,NaN
3,2022-01-31,TL7010807,1.860062e+06,NAN,OFFSHORE,NaN,NaN,NaN
4,2022-02-25,TL7003471,-1.127797e+03,NAN,OFFSHORE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
10796,2025-07-29,8350586,5.000000e+04,CAIC ZEM GOMES,DEPÓSITO EM C/C VIA CIP - KAFRE ALIMENTOS LTDA,CONTA CORRENTE,Conta Corrente,CRÉDITO
10797,2025-07-29,8864484,2.000000e+04,RODRIGO DE MELLO D’ELIA,TRANSFERÊNCIA A CRÉDITO VIA PIX - SEVERI EMPRE...,CONTA CORRENTE,Conta Corrente,CRÉDITO
10798,2025-07-29,9080245,3.535825e+05,FERNANDO DOMINGUES DA SILVA,DEPÓSITO - BRADESCO PN N1,RENDA VARIÁVEL,BBDC4,DEPÓSITO
10799,2025-07-29,9080245,2.266719e+04,FERNANDO DOMINGUES DA SILVA,DEPÓSITO - ISHARE SP500CI,RENDA VARIÁVEL,IVVB11,DEPÓSITO


##### MIGRAÇÕES

In [8]:
conexao = Connect.connect_techdb()
migracoes_btg = Connect.import_table(conexao, "migracoes_btg")
troca_assessores = pd.read_excel(diretorio_base+"%s\\troca_assessor.xlsx" % (data))
migracoes_btg = migracoes_btg[
    (migracoes_btg['CONTA'].isin(troca_assessores['Conta'].astype(str))) |
    (migracoes_btg['CONTA'] == '590732') |
    (migracoes_btg['CONTA'] == '299305') |
    (migracoes_btg['CONTA'] == '5173757') |
    (migracoes_btg['CONTA'] == '5152837') |
    (migracoes_btg['CONTA'] == '5149832') |
    (migracoes_btg['CONTA'] == '5917705') |
    (migracoes_btg['CONTA'] == '15296593')
]
conexao.close()
captacao['TIPO DE CAPTACAO'] = "Padrão"
migracoes_btg['TIPO DE CAPTACAO'] = "Migração BTG"
migracoes_btg['DESCRIÇÃO'] = "Migração BTG"
migracoes_btg['MERCADO'] = "Migração BTG"

c:\Scripts\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
captacao = pd.concat([captacao, migracoes_btg], axis=0)
captacao = captacao[captacao['TIPO'] != "RS"]
captacao.dropna(subset='DATA', axis=0, inplace=True)
captacao['CONTA'] = captacao['CONTA'].astype(str)
captacao['CAPTAÇÃO'] = captacao['CAPTAÇÃO'].astype(float)
captacao['DATA'] = pd.to_datetime(captacao['DATA'], format="%Y-%m-%d %H:%M:%S")
captacao = captacao[['DATA', 'CONTA', 'CAPTAÇÃO', 'Assessor', 'TIPO DE CAPTACAO', 'MERCADO']]

In [10]:
captacao =captacao[captacao['DATA'] >= quinze_dias].sort_values(by='DATA', ascending=False)

##### CAPTAÇÃO HISTORICO

In [11]:
conexao = Connect.connect_techdb()
captacao_historico = Connect.import_table(conexao, "captacao_historico")
conexao.close()

In [12]:
captacao_historico = captacao_historico[captacao_historico['DATA'] < quinze_dias]

In [13]:
conexao = Connect.connect_techdb()
primeiro_vinculo = Connect.import_table(conexao, "primeiro_vinculo")
primeiro_vinculo.rename(columns={"Conta":"CONTA"}, inplace=True)
conexao.close()

In [14]:
basebtg = Bases.basebtg()

In [15]:
### remover aportes antes do primeiro vinculo
#captacao_historico = captacao_historico.merge(primeiro_vinculo[['CONTA', 'Data Vínculo']], on=['CONTA'], how='left')
#anteriores_ao_vinculo = captacao_historico[(captacao_historico['DATA'] < captacao_historico["Data Vínculo"]) & (captacao_historico['TIPO DE CAPTACAO'] != "Migração BTG")]
#captacao_historico = captacao_historico[~captacao_historico.isin(anteriores_ao_vinculo)]
#captacao_historico.drop("Data Vínculo", axis=1, inplace=True)

In [16]:
captacao_historico = pd.concat([captacao, captacao_historico], axis=0)

In [17]:
situacao = []
for conta in captacao_historico['CONTA']:
    if conta in list(basebtg['Conta']):
        situacao.append("Ativo")
    else:
        situacao.append("Inativo")

captacao_historico['Situacao'] = situacao

In [18]:
conexao = Connect.connect_techdb()
pl_historico = conexao.execute(text('''
                               SELECT "Conta", "PL Total", "Data" FROM principal."pl_historico_diario";
                               '''))
conexao.close()
pl_historico = pd.DataFrame(pl_historico, columns=[x for x in pl_historico.keys()])
pl_historico.rename(columns={"Data":"DATA", "Conta":"CONTA"}, inplace=True)
pl_historico.drop_duplicates(subset=["CONTA", "DATA"], keep='first', inplace=True)

In [19]:
contas_inativas = captacao_historico[
        (captacao_historico['Situacao'] == "Inativo")
    ].drop_duplicates(subset='CONTA')

In [20]:
debitos = []

for data, conta, assessor, situacao in zip(
                                            contas_inativas['DATA'], contas_inativas['CONTA'].astype(str),
                                            contas_inativas['Assessor'], contas_inativas['Situacao']
                                           ):
    if situacao == "Inativo":
        try:
            captacao = pl_historico[pl_historico['CONTA'] == conta].sort_values("DATA").iloc[-1:,:]['PL Total'].values[0] * -1
        except:
            captacao = 0
        data = pl_historico[pl_historico['CONTA'] == conta]['DATA'].max()
        
        df = pd.DataFrame({
            "DATA":[data], "CONTA":[conta], "CAPTAÇÃO":[captacao], "Assessor":[assessor], "Situacao":[situacao]
            })
        debitos.append(df)
    else:
        print("ERRO: ", conta)
        
        
debitos = pd.concat(debitos).drop_duplicates(subset = 'CONTA', keep = 'first')

In [21]:
debitos = debitos[debitos['DATA'] >= quinze_dias]

In [22]:
conexao = Connect.connect_techdb()
entradas_e_saidas = Connect.import_table(conexao, "Entradas_e_saidas_consolidado")
entradas_e_saidas = entradas_e_saidas[['Conta', 'Mês de entrada/saída']]
entradas_e_saidas.rename(columns={"Conta":"CONTA"}, inplace=True)
entradas_e_saidas = entradas_e_saidas.drop_duplicates("CONTA", keep='last')
conexao.close()
debitos = pd.merge(debitos, entradas_e_saidas, on='CONTA', how='left')
debitos['Mês de entrada/saída'] = pd.to_datetime(debitos['Mês de entrada/saída'], format="%d-%m-%Y", errors='coerce')
debitos.loc[debitos['Mês de entrada/saída'].isnull(), "Mês de entrada/saída"] = debitos[debitos['Mês de entrada/saída'].isnull()]['DATA']
debitos.drop('DATA', axis=1, inplace=True)
debitos.rename(columns={"Mês de entrada/saída":"DATA"}, inplace=True)
debitos = debitos[debitos['DATA'].isna() == False]
debitos['TIPO DE CAPTACAO'] = "Saída de conta"
debitos['MERCADO'] = "Saída de conta"

In [23]:
captacao_historico = pd.concat([captacao_historico, debitos], axis=0)

In [24]:
################## ATUALIZAR ASSESSOR ATUAL #############
assessor_atual = basebtg[['Conta', 'Assessor']]
assessor_atual.rename(columns={"Conta":"CONTA", "Assessor":"Assessor atual"}, inplace=True)
captacao_historico = captacao_historico.merge(assessor_atual, on='CONTA', how='left')
captacao_historico.loc[captacao_historico['Assessor atual'].isnull(), "Assessor atual"] = captacao_historico[captacao_historico['Assessor atual'].isnull()]['Assessor']
captacao_historico.drop("Assessor", axis=1, inplace=True)
captacao_historico.rename(columns={"Assessor atual":"Assessor"}, inplace=True)

C:\Users\joaob\AppData\Local\Temp\ipykernel_21976\1020268685.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assessor_atual.rename(columns={"Conta":"CONTA", "Assessor":"Assessor atual"}, inplace=True)


In [25]:
captacao_historico['DATA'] = pd.to_datetime(captacao_historico['DATA'])

In [26]:
nomes_clientes = pd.read_excel(r"C:\\Scripts\\nomes_clientes\Nomes_Clientes2.xlsx")

In [27]:
captacao_historico.drop("Nome", axis=1, inplace=True)
nomes = nomes_clientes
nomes.drop_duplicates("Conta", inplace=True)
nomes.rename(columns={"Conta":"CONTA"}, inplace=True)
captacao_historico = captacao_historico.merge(nomes, on='CONTA', how='left')

In [28]:
conexao = Connect.connect_techdb()
captacao_historico.to_sql("captacao_historico", 
                          con=conexao, 
                          index=False, 
                          if_exists='replace', 
                          dtype={'CONTA':sqlalchemy.VARCHAR(255)},
                          schema='principal')
conexao.close()